In [31]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import pandas_ta as ta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import seaborn as sns

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()

# feature engineering and scaling
def prep(dataset):
    scaler = StandardScaler()

    macro_path1 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\IRP DATASET  - Copy of US_macroeconomics.csv"
    macro_path2 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\^VIX.csv"
    macro_path3 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\CORESTICKM159SFRBATL.csv"
    macro_path4 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\FFR.csv"
    macro_path5 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GDP.csv"
    macro_path6 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VIX9D_History.csv"
    macro_path7 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GVZ_History.csv"
    macro_path8 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\OVX_History.csv"
    macro_path9 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VVIX_History.csv"
    macro_data1 = pd.read_csv(macro_path1)
    macro_data2 = pd.read_csv(macro_path2)
    macro_data3 = pd.read_csv(macro_path3)
    macro_data4 = pd.read_csv(macro_path4)
    macro_data5 = pd.read_csv(macro_path5)
    macro_data6 = pd.read_csv(macro_path6)
    macro_data7 = pd.read_csv(macro_path7)
    macro_data8 = pd.read_csv(macro_path8)
    macro_data9 = pd.read_csv(macro_path9)

    dataset['Date'] = pd.to_datetime(dataset['Date'], dayfirst=True)
    dataset.set_index('Date', inplace=True)
    dataset['Target'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

    macro_data1['date'] = pd.to_datetime(macro_data1['date'], dayfirst=True)
    macro_data1.set_index('date', inplace=True)

    macro_data2['Date'] = pd.to_datetime(macro_data2['Date'], dayfirst=True)
    macro_data2.set_index('Date', inplace=True)

    macro_data3['Date'] = pd.to_datetime(macro_data3['Date'], dayfirst=True)
    macro_data3.set_index('Date', inplace=True)

    macro_data4['Date'] = pd.to_datetime(macro_data4['Date'], dayfirst=True)
    macro_data4.set_index('Date', inplace=True)

    macro_data5['Date'] = pd.to_datetime(macro_data5['Date'], dayfirst=True)
    macro_data5.set_index('Date', inplace=True)

    macro_data7['Date'] = pd.to_datetime(macro_data7['Date'], dayfirst=False)
    macro_data7.set_index('Date', inplace=True)

    macro_data8['Date'] = pd.to_datetime(macro_data8['Date'], dayfirst=False)
    macro_data8.set_index('Date', inplace=True)

    macro_data9['Date'] = pd.to_datetime(macro_data9['Date'])
    macro_data9.set_index('Date', inplace=True)

    merge_data1 = dataset.join(macro_data1, how='left').fillna(method='ffill')
    merge_data2 = merge_data1.join(macro_data2, how='left').fillna(method='ffill')
    merge_data3 = merge_data2.join(macro_data3, how='left').fillna(method='ffill')
    merge_data4 = merge_data3.join(macro_data4, how='left').fillna(method='ffill')
    merge_data5 = merge_data4.join(macro_data5, how='left').fillna(method='ffill')
    merge_data7 = merge_data5.join(macro_data7, how='left').fillna(method='ffill')
    merge_data8 = merge_data7.join(macro_data8, how='left').fillna(method='ffill')
    merge_final = merge_data8.join(macro_data9, how='left').fillna(method='ffill')

    merge_final['RSI (14D)'] = ta.rsi(merge_final['Close'], length=14)
    merge_final['20 Day CCI'] = ta.cci(high=merge_final['High'], low=merge_final['Low'], 
                                    close=merge_final['Close'], length=20)
    merge_final['Williams %R'] = ta.willr(high=merge_final['High'], low=merge_final['Low'], 
                                        close=merge_final['Close'], length=14)
    merge_final['EMA (5D)'] = merge_final['Close'].ewm(span=5, adjust=False).mean()
    merge_final['MA50'] = merge_final['Close'].rolling(window=50).mean()

    final_dataset = merge_final.dropna()

    time_shifted = final_dataset.resample('M').asfreq().dropna()
    
    features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'DFF']
    
    time_shifted[features] = time_shifted[features].astype(float)
    time_shifted[features] = scaler.fit_transform(time_shifted[features])

    return time_shifted, scaler

# get dataset
file_path = filedialog.askopenfilename(parent=root, title="Select A File")
ticker = pd.read_csv(file_path)
ticker, scaler = prep(ticker)

ticker

,Open,High,Low,Close,Adj Close,Volume,Target,CPI,Mortgage_rate,Unemp_rate,...,DFF,GDP,GVZ,OVX,VVIX,RSI (14D),20 Day CCI,Williams %R,EMA (5D),MA50
Date,,,,,,,,,,,,,,,,,,,,,
2009-09-30,-1.327600,-1.329800,-1.336358,-1.329921,-1.329921,1.787928,0.0,-1.481362,1.465477,1.650422,...,-0.626910,14448.882,0.918290,0.448117,-1.501036,0.335941,0.096770,-0.428568,-1.329277,-1.357324
2009-11-30,-1.301656,-1.300737,-1.301760,-1.296622,-1.296622,-0.390549,1.0,-1.452538,1.302385,1.735954,...,-0.589478,14651.249,1.928935,0.209814,-0.798962,-0.109199,-0.034281,-0.736219,-1.293855,-1.309026
2009-12-31,-1.270980,-1.274628,-1.276920,-1.279804,-1.279804,-2.274042,1.0,-1.415262,1.268375,1.693188,...,-0.639387,14651.249,1.381947,-0.078379,-1.249904,-0.083951,0.365310,-0.119932,-1.274612,-1.291614
2010-03-31,-1.231999,-1.234331,-1.232598,-1.232874,-1.232874,0.219305,1.0,-1.414995,1.326346,1.693188,...,-0.614432,14651.249,0.191822,-0.468702,-1.078834,1.114002,0.445119,0.320229,-1.232367,-1.269525
2010-04-30,-1.201763,-1.205620,-1.214725,-1.217966,-1.217966,1.839092,1.0,-1.412771,1.528086,1.693188,...,-0.545807,14980.193,0.678983,-0.431137,0.557969,-0.531879,-0.600103,-1.619357,-1.209660,-1.229610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,1.688896,1.681724,1.699686,1.702582,1.702582,1.166950,1.0,1.757019,1.349535,-1.001065,...,2.654622,25994.639,-0.645753,0.105924,-0.447926,1.555560,0.481108,0.996341,1.696813,1.558154
2024-01-31,1.986173,1.971033,1.967486,1.942584,1.942584,0.438650,1.0,1.757019,1.349535,-1.001065,...,2.654622,25994.639,-0.872240,-0.020271,-0.389762,0.282188,0.350883,-0.041148,1.980572,1.869774
2024-02-29,2.146907,2.141290,2.155992,2.159066,2.159066,0.980975,1.0,1.757019,1.349535,-1.001065,...,2.654622,25994.639,-1.299574,-0.357768,-1.081571,1.172257,0.849979,1.020216,2.146899,2.016940


In [32]:
# create LSTM model class
class LSTM_Model(nn.Module):
    def __init__(self, input_layer, hidden_layer, output_layer, num_layers=4, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.hidden_layer = hidden_layer
        self.num_layers = num_layers
        
        # LSTM layers with dropout
        self.lstm = nn.LSTM(input_layer, hidden_layer, num_layers, batch_first=True, dropout=dropout)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Linear layer
        self.linear_layer = nn.Linear(hidden_layer, output_layer)

    def forward(self, input_tensor):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        c0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        
        # Pass through LSTM layers
        out, _ = self.lstm(input_tensor, (h0, c0))
        
        # Apply dropout
        out = self.dropout(out)
        
        # Pass through linear layer
        out = self.linear_layer(out[:, -1, :])
        return out

# create sequences for input data and corresponding labels
def create_sequence(input_data, sequence_length):
    sequences = []
    for i in range(0, len(input_data) - sequence_length):
        sequence = input_data[i : i + sequence_length, :-1]
        label = input_data[i + sequence_length, -1]
        sequences.append((sequence, label))
    return sequences

# train the model with data provided
def trainer(model, train_data, loss_func, opt, epochs, fold, losses):
    model.train()
    for epoch in range(epochs):
        for sequence, labels in train_data:
            opt.zero_grad()
            sequence = sequence.clone().detach().float()
            labels = labels.clone().detach().float().view(-1, 1)
            
            y = model(sequence)
            loss = loss_func(y, labels)
            loss.backward()
            opt.step()

        if epoch % 25 == 1:
            print(f'Epoch {epoch} loss: {loss.item():.4f}')

            new_row = pd.DataFrame({'Fold': [fold+1], 'Epoch': [epoch], 'Loss': [loss.item()]})
            losses = pd.concat([losses, new_row], ignore_index=True)
            
    return losses

def predictor(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for sequence, _ in test_data:
            sequence = sequence.clone().detach().float()
            y = model(sequence)
            batch_predictions = torch.sigmoid(y)

            # Convert predictions to tensor if they are scalar or float
            if isinstance(batch_predictions, float) or batch_predictions.ndimension() == 0:
                batch_predictions = torch.tensor([batch_predictions])

            batch_predictions = torch.round(batch_predictions)

            predictions.extend(batch_predictions.squeeze().tolist())
    
    return predictions

# create sequences
sequence_length = 10
sequences = create_sequence(ticker[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                                    'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'Target']].values, sequence_length)

# cross-validation
tscv = TimeSeriesSplit(n_splits=10)
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []
losses = pd.DataFrame(columns=['Fold', 'Epoch', 'Loss'])

for fold, (train_index, test_index) in enumerate(tscv.split(sequences)):
    print(f'Fold {fold+1}')
    
    train_sequences = [sequences[i] for i in train_index]
    test_sequences = [sequences[i] for i in test_index]
    
    train_data = torch.utils.data.DataLoader(train_sequences, shuffle=True, batch_size=32)
    test_data = torch.utils.data.DataLoader(test_sequences, shuffle=False, batch_size=32)
    
    # initialize model
    model = LSTM_Model(input_layer=12, hidden_layer=10, output_layer=1)
    loss_func = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    # train model
    epochs = 150
    losses = trainer(model, train_data, loss_func, opt, epochs, fold, losses)
    
    # make predictions
    test_labels = [label for _, label in test_sequences]
    predictions = predictor(model, test_data)
    
    # calculate statistics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, zero_division=0)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    kappa = cohen_kappa_score(test_labels, predictions)
    
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)
    
    print(f'\nAccuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1: {f1:.4f}')
    print(f'Kappa: {kappa:.4f}\n')

# average scores across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_kappa = np.mean(kappas)

print(f'\nCross-Validation Results:')
print(f'Average Accuracy: {avg_accuracy:.4f}')
print(f'Average Precision: {avg_precision:.4f}')
print(f'Average Recall: {avg_recall:.4f}')
print(f'Average F1: {avg_f1:.4f}')
print(f'Average Kappa: {avg_kappa:.4f}')


Fold 1
Epoch 1 loss: 0.7067
Epoch 26 loss: 0.6663
Epoch 51 loss: 0.6592
Epoch 76 loss: 0.6785
Epoch 101 loss: 0.6776
Epoch 126 loss: 0.6989

Accuracy: 0.2000
Precision: 0.0000
Recall: 0.0000
F1: 0.0000
Kappa: 0.0000

Fold 2
Epoch 1 loss: 0.6816
Epoch 26 loss: 0.6811
Epoch 51 loss: 0.6873
Epoch 76 loss: 0.6869
Epoch 101 loss: 0.6836
Epoch 126 loss: 0.6792

Accuracy: 0.2000
Precision: 0.2000
Recall: 1.0000
F1: 0.3333
Kappa: 0.0000

Fold 3
Epoch 1 loss: 0.7031
Epoch 26 loss: 0.7136
Epoch 51 loss: 0.6852
Epoch 76 loss: 0.6900
Epoch 101 loss: 0.7292
Epoch 126 loss: 0.6780

Accuracy: 0.4000
Precision: 0.0000
Recall: 0.0000
F1: 0.0000
Kappa: 0.0000

Fold 4
Epoch 1 loss: 0.7024
Epoch 26 loss: 0.6907
Epoch 51 loss: 0.6838
Epoch 76 loss: 0.6933
Epoch 101 loss: 0.6911
Epoch 126 loss: 0.6889

Accuracy: 0.4000
Precision: 0.4000
Recall: 0.4000
F1: 0.4000
Kappa: -0.2000

Fold 5
Epoch 1 loss: 0.7163
Epoch 26 loss: 0.6941
Epoch 51 loss: 0.6923
Epoch 76 loss: 0.6838
Epoch 101 loss: 0.6796
Epoch 126 loss

In [33]:
fall = (ticker.Target == 0).sum()
rise = (ticker.Target == 1).sum()

rise_per = (rise / (rise + fall)) * 100
fall_per = (fall / (rise + fall)) * 100

print(f'Number of days rising: {rise}')
print(f'Number of days falling: {fall}')

print(f'Rise % is: {rise_per:.2f}%')
print(f'Fall % is: {fall_per:.2f}%')

Number of days rising: 75
Number of days falling: 50
Rise % is: 60.00%
Fall % is: 40.00%


In [34]:
results = pd.DataFrame(columns=['Dataset', 'Average Accuracy', 'Average Precision', 
                                'Average Recall', 'Average F1', 'Average Kappa', 'Number of Rises', 
                                'Number of Falls', 'Rise %', 'Fall %'])

datatset_name = (file_path.split('/')[-1].split('.')[0] + '_Monthly')

metrics = {'Dataset': datatset_name ,'Average Accuracy': avg_accuracy, 
           'Average Precision': avg_precision, 'Average Recall': avg_recall, 
           'Average F1': avg_f1, 'Average Kappa': avg_kappa, 'Number of Rises': rise, 'Number of Falls': fall, 
           'Rise %': rise_per, 'Fall %': fall_per}

export_results = results._append(metrics, ignore_index=True)

export_results = pd.concat([export_results, losses], ignore_index=True)

export_results.set_index('Dataset', inplace=True)

export_results.to_csv(r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\IRP DATA\\" + datatset_name + '_monthly.csv')

export_results

,Average Accuracy,Average Precision,Average Recall,Average F1,Average Kappa,Number of Rises,Number of Falls,Rise %,Fall %,Fold,Epoch,Loss
Dataset,,,,,,,,,,,,
^SPX_Monthly,0.48,0.42,0.59,0.452908,0.008571,75,50,60.0,40.0,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.706665
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,26,0.666313
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,51,0.659155
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,76,0.678531
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,26,0.711847
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,51,0.686141
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,76,0.626328
